Notebook inspired by Andrej Karpathy's film: https://www.youtube.com/watch?v=kCc8FmEb1nY&

In [1]:
import gdown  # aby pobrać zbiór danych z dysku google
import os.path

from torch import tensor
import torch

Zbiór danych jest dowolny tak długo, jak długo jest tekstowy. 
Na początek naszy zbiór będzie bardzo prosty - jeden plik tekstowy zawierający dzieła Szekspira.

Zbiór danych jest pobierany z dysku google (jeśli nie jest już pobrany) a następnie wczytywany jako zwykły plik tekstowy

In [2]:
dataset_link = "https://drive.google.com/uc?id=1TQjhbN1jrQx7eMgySFkMfwahh7IZy2a8"
dataset_path = "data/Shakespeare.txt"
if not os.path.isfile(dataset_path):
    print("Downloading dataset...")
    gdown.download(dataset_link, dataset_path)
    print("Done!")
else:
    print("Dataset is already downloaded")
    
with open(dataset_path, "r", encoding="utf-8") as f:
    whole_text = f.read()

Dataset is already downloaded


In [3]:
print(f"Length of the dataset: {len(whole_text)}")

Length of the dataset: 1115394


"Podejrzymy" początek naszego zbioru danych

In [4]:
print(whole_text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



Nasz LLM, jak każda inna sieć neuronowa, operuje na liczbach. Potrzebny nam jest enkoder, który pozwoli zamienić tekst na liczby oraz dekoder, który odwróci działanie enkodera.

Aby zdefiniować jakikolwiek enkoder potrzebny nam jest kompletny zbiór słownictwa. Aby zachować prostotę, ograniczymy się tutaj do pojedyńczych liter - jedna litera będzie jednym tokenem. Niemniej należy pamiętać, że w bardziej zaawansowanych modelach dużo lepiej działają tokeny składające się ze zlepków liter.

Podział tekstu na tokeny jest wielkim problemem samym w sobie i nie będziemy się w niego zagłębiać w ramach tego projektu

In [5]:
vocab = sorted(list(set(whole_text)))  # set zapewnia unikalność znaków, lista daje się posortować
vocab_size = len(vocab)
print(f"Vocab: {"".join(vocab)}")
print(f"Vocab len: {len(vocab)}")

Vocab: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab len: 65


In [6]:
stoi = {char: i for i, char in enumerate(vocab)}
itos = {i: char for i, char in enumerate(vocab)}
encode = lambda s: [stoi[c] for c in s]  # zamienia string na listę liczb
decode = lambda l: "".join(itos[i] for i in l)

print(encode("Hello world!"))
print(decode(encode("Hello world!")))

[20, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]
Hello world!


Wczytajmy cały zbiór danych (w końcu waży tylko 1MB) do tensora

In [7]:

data = tensor(encode(whole_text))
print(f"{data.shape = }, {data.dtype = }")
print(data[:100])

data.shape = torch.Size([1115394]), data.dtype = torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


Podzielimy nasz zbiór danych na część testową i walidacyjną aby móc ocenić, na ile dobrze model uczy się ogólnych tendencji, a nie po prostu "zapamiętuje".  Zazwyczaj zbiory danych dzielone są losowo, tak, aby żadna z "części" takiego zbioru nie była słabiej reprezentowana. 

W tym przypadku wyciąganie wyrazów ze środka tekstu nie będzie dobrą metodą, ponieważ w analizie tekstu kluczowy jest kontekst. W związku z tym tekst zostanie "przedzielony" na dwie części

In [8]:
train_to_all_ratio = 0.85
n = int(train_to_all_ratio * data.numel())
train_data, test_data = data[:n], data[n:]
print(f"{len(train_data) = }, {len(test_data) = }")

len(train_data) = 948084, len(test_data) = 167310


Przy ćwiczeniu modeli językowych nie można podać całego zbioru danych na raz. Tekst jest podawany w losowo wybieranych blokach.
Wielkość takiego bloku jest ustalona i jest jednocześnie **maksymalnym** rozmiarem kontekstu dostępnego dla naszego modelu. Dla każdego bloku model jest ćwiczony dla wszystkich kontekstów zaczynając od 1 a kończąc na `block_size`.

Zobrazujmy to za pomocą pętli:

In [9]:
context_size = 8
x = train_data[:context_size]
y = train_data[1:context_size + 1]  # cel jest przesunięty o 1 względem danej treningowej
for i in range(context_size):
    context = x[:i+1]  # do i-tego znaku włącznie
    target = y[i]
    print(f"For input {context} the target is {target}")

For input tensor([18]) the target is 47
For input tensor([18, 47]) the target is 56
For input tensor([18, 47, 56]) the target is 57
For input tensor([18, 47, 56, 57]) the target is 58
For input tensor([18, 47, 56, 57, 58]) the target is 1
For input tensor([18, 47, 56, 57, 58,  1]) the target is 15
For input tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
For input tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


Ćwiczenie modelu na wszystkich rozmiarach kontekstu, oprócz oczywistej zalety w postaci szybkości uzyskiwania danych (sekwencja tekstu i tak jest pozyskana), pozwala nauczyć model działania także przy krótszych kontekstach. W naszym przypadku kontekst nie będzie duży, jednak w przypadku takich modeli jak GPT-3.5 czy GPT-4 rozmiar kontekstu może wynosić nawet setki tysięcy tokenów i większość zapytań nie będzie tak długa

Karty graficzne są bardzo dobre w zrównoleglaniu obliczeń - dlatego podczas korzystania z modeli podaje im się wiele niezależnych wartości, które są przetwarzane jednocześnie. Zbiór wszystkich takich wartości nazywany jest batchem

In [10]:
batch_size = 64

rng = torch.Generator()
rng.manual_seed(42)

def get_random_batch(split: str, batch_size:int, block_size: int, rng: torch.Generator) -> (torch.Tensor, torch.Tensor):
    """
    Returns a random batch of data - tensor of shape (batch_size, block_size). 
    :param split: can be either "train" or "test". When train the train dataset is used
    :returns: A tuple of two tensors - first with training data and second with labels/targets
    """
    if split == "train": my_data = train_data
    elif split == "test": my_data = test_data
    else: raise ValueError(f"Expected either `train` or `test` for the split argument, got {split} instead")
    idx = torch.randint(len(my_data) - block_size, size=(batch_size,), generator=rng)
    # randint jest end-exclusive, dlatego nie trzeba modyfikować indeksów mimo że target jest i+1
    x = torch.stack([data[i: i+block_size] for i in idx])
    y = torch.stack([data[i+1: i+block_size+1] for i in idx])
    return x, y
    
xb, yb = get_random_batch("train", batch_size, context_size, rng)
print(f"Data shape: {xb.shape}")
print(f"Label shape: {yb.shape}")

Data shape: torch.Size([64, 8])
Label shape: torch.Size([64, 8])


Zwracam uwagę, że tutaj dane treningowe oraz targety mają ten sam kształt - dla każdego jednego znaku chcemy wiedzieć, jaki znak jest następny.

Można zauważyć, że jest to niepotrzebne duplikowanie informacji, ponieważ dla wszystkich znaków oprócz ostatniego jest ona zawarta już w danych treningowych. To prawda, jednak przy ilości obliczeń wykonywanych przez transformery nie robi to zauważalnej różnicy w wydajności treningu, a kod jest bardziej czytelny